In [1]:
using DataFrames
using JuMP, Gurobi

In [2]:
# number of teams
N = 20
# number of weeks
W = N-1
# number of days per weeks
D = 7;

# Define functions

In [3]:
function GetC()

    W_ = 19

    C = zeros((D*(W_+1), D*(W_+1)))

    for t in 1:(W_*D)
        for t_ in 1:(W_*D)
            if t < t_
                C[t, t_] = (t_ - t)^(-1)
            end
        end
    end

    return C
end

GetC (generic function with 1 method)

In [4]:
function calendar_week(x,week)
    df = DataFrame(Player=1:N, Tuesday=0.0, Wednesday=0.0, Thursday=0.0, Friday=0.0, Saturday=0.0, Sunday=0.0, Monday=0.0)
    for i=1:N
        for j=1:N
            for d=1:D
                if x[i,j,week,d] == 1.0
                        df[i, 1+d] = j
                end
            end
        end
    end
    return df
end

calendar_week (generic function with 1 method)

In [72]:
# compute the value of the objective function based on the calendar
function ComputeObjectiveValue(calendar, W)

    # define empty matrix to concatenate all the other weeks
    m = Array{Float64}(undef, 20, 0)
    
    # build complete calendar
    for w in 1:W
        week = Matrix(calendar[w])
        m = [m week]
    end

    return ComputeObjectiveValueMatrix(m, W)
end

ComputeObjectiveValue (generic function with 1 method)

In [73]:
# compute the value of the objective function based on the calendar
function ComputeObjectiveValueMatrix(m, W)

    # define total cost for each team
    team_total_cost = []

    # for each team 
    for i = 1:N
        # find index of nonzer elements, they correspond to the days in which the team plays
        team_plays = findall(x -> x != 0, m[i, :])

        # we will find W non-zero elements, since each team plays against all the other teams
        # for i = 1:W, return GetC()[team_plays[i], team_plays[i+1]] and sum
        append!(team_total_cost, sum([GetC()[team_plays[i], team_plays[i+1]] for i = 1:W-1]))
    end

    # total cost of the calendar
    return sum(team_total_cost)
end

ComputeObjectiveValueMatrix (generic function with 1 method)

In [74]:
function GetCalendarMatrix(calendar, W)

    # define empty matrix to concatenate all the other weeks
    m = Array{Float64}(undef, 20, 0)
    
    # build complete calendar
    for w in 1:W
        week = Matrix(calendar[w])[:, 2:end] # first column is the index
        m = [m week]
    end

    return m
end

GetCalendarMatrix (generic function with 1 method)

# Model itself

In [75]:
function Optimize4WeeksAndFreeze(Y_old, Win, Wend)

    # create model
    model = Model(Gurobi.Optimizer)

    # set time limit
    set_optimizer_attribute(model, "TimeLimit", 600)

    # dont print output
    set_silent(model)

    # PARAMETERS
    C = GetC()
    N = 20 # teams
    D = 7 # days

    # VARIABLES
    @variable(model, x[i = 1:N, j = 1:N, w = Win:Wend, d = 1:D], Bin) # 1 if team i plays team j on day d of week w, 0 otherwise
    @variable(model, a[i = 1:N, j = 1:N, k = 1:N, w = Win:Wend, d = 1:D, d_ = 1:D] >= 0) # will be pushed to be binary
    @variable(model, y[i = 1:N, j = 1:N], Bin) # 1 if team i played team j on w, 0 otherwise

    # OBJECTIVE FUNCTION
    # put penalty on having y[i,j] = 1, otherwise formulation put all of them to 1
    @objective(model, Min, sum(C[7 * w + d, 7 * (w+1) + d_] * a[i, j, k, w, d, d_] for i in 1:N, 
        j in 1:N, k in 1:N, w in Win:Wend-1, d in 1:D, d_ in 1:D) + sum(y[i, j] for i = 1:N, j = 1:N))

    # CONSTRAINTS
    # linearize objective function
    @constraint(model, [i = 1:N, j = 1:N, k = 1:N, w = Win:Wend-1, d = 1:D, d_ = 1:D], a[i, j, k, w, d, d_] <= x[i, j, w, d])
    @constraint(model, [i = 1:N, j = 1:N, k = 1:N, w = Win:Wend-1, d = 1:D, d_ = 1:D], a[i, j, k, w, d, d_] <= x[i, k, w+1, d])
    @constraint(model, [i = 1:N, j = 1:N, k = 1:N, w = Win:Wend-1, d = 1:D, d_ = 1:D], a[i, j, k, w, d, d_] >= x[i, j, w, d] + x[i, k, w+1, d] - 1)

    # teams can never play against themselves
    @constraint(model, [i = 1:N, w = Win:Wend, d = 1:D], x[i, i, w, d] == 0)

    # if A plays against B, then B plays against A
    @constraint(model, [i = 1:N, j = 1:N, w = Win:Wend, d = 1:D], x[i, j, w, d] == x[j, i, w, d])

    # each team can only play at most one game per week
    @constraint(model, [i = 1:N, w = Win:Wend], sum(x[i, j, w, d] for j in 1:N, d in 1:D) <= 1)

    # each team plays Wend - Win + 1 games in total
    @constraint(model, [i = 1:N], sum(x[i, j, w, d] for j in 1:N, w in Win:Wend, d in 1:D) == Wend - Win + 1)

    # each game is against a different team, team play against each other team at most once
    @constraint(model, [i = 1:N, j = 1:N], sum(x[i, j, w, d] for w in Win:Wend, d in 1:D) <= 1)

    # if team i plays team j, update Y
    @constraint(model, [i = 1:N, j = 1:N], y[i, j] >= sum(x[i, j, w, d] for d in 1:D, w = Win:Wend))

    # if team i played team j, team j played team i
    @constraint(model, [i = 1:N, j = 1:N], y[i, j] == y[j, i])
    
    # if team i played team j in the past, then they can't play again now
    @constraint(model, [i = 1:N, j = 1:N], sum(x[i,j,w,d] for w in Win:Wend, d in 1:D) <= 1 - Y_old[i, j])

    # cannot plays on days 1,2,3
    @constraint(model, [i = 1:N, j = 1:N, w = Win:Wend, d = 1:3], x[i, j, w, d] == 0)

    # OPTIMIZE
    optimize!(model)

    # update old X, Y before returning them
    Y_old = Y_old .+ value.(y)

    return value.(x), Y_old

end


Optimize4WeeksAndFreeze (generic function with 1 method)

# Call model

In [50]:
Win_list = [1, 5, 9, 13, 17]
Wend_list = [4, 8, 12, 16, 19]

# first time, Y_old initialized to empty
Y_old = zeros(20, 20)   # teams never played against each other before

# define calendar list, each time append the calendar week to 
calendar = []
;

In [ ]:
for (Win, Wend) in zip(Win_list, Wend_list)

    println("Initial week: ", Win, ", end week: ", Wend)
    println("#############################################")

    # get value of X and Y
    X, Y_old = Optimize4WeeksAndFreeze(Y_old, Win, Wend)

    for w in Win:Wend
        push!(calendar, calendar_week(X, w))
    end

end

In [76]:
# in this matrix all elements except the diagonal should be 1, in the diagonal they should all be 0
println(sum([Y_old[i,i] for i in 1:20]))

# all other elements should be 1 (400 elements in total, diagonal = 20 element is 0, sum should be 380)
println(sum(Y_old))

0.0
0.0


In [78]:
obj_calendar = ComputeObjectiveValue(calendar, W)

13.619047619047615

In [ ]:
for w in 1:19
    display(calendar[w])
end

# Random restart

Idea is simple, take the current solution, pick randomly either 2, 3 or 4 consecutive weeks and re-optimize just them. If the objective function improves, keep it, otherwise discard it. Repeat for a certain number of iterations.

In [ ]:
function random_restart(calendar_matrix, obj_calendar, Y_old)
    
    # randomly pick a number of weeks to reoptimize (in addition to initial week)
    n = rand(1:4)

    # randomly choose initial week
    Win_ = rand(1:19-n)
    Wend_ = Win_ + n

    # get portion of the calendar matrix corresponding the the weeks to reoptimize
    calendar_reoptimize = calendar_matrix[:, 7 * (Win_-1) + 1 : 7 * Wend_]

    # for each team, get the list of opponents in these weeks
    for i in 1:20
        plays_against = [Int(element) for element in calendar_reoptimize[i, :] if element != 0]
        # modify Y_old accordingly, putting it to 0 so that the teams can be made play again
        for element in plays_against
            Y_old[i, element] = 0
        end
    end

    # call the model
    X_, Y_old = Optimize4WeeksAndFreeze(Y_old, Win_, Wend_)

    # get new calendar
    calendar_reoptimize = []

    for w in Win_:Wend_
        push!(calendar_reoptimize, calendar_week(X_, w))
    end

    calendar_reoptimize = GetCalendarMatrix(calendar_reoptimize, Wend_ - Win_ + 1)

    # update calendar
    calendar_new = calendar_matrix
    calendar_new[:, 7 * (Win_-1) + 1 : 7 * Wend_] = calendar_reoptimize

    # find objective value corresponding to new calendar
    obj_calendar_new = ComputeObjectiveValueMatrix(calendar_new, W)

    println("Objective value of new calendar: ", obj_calendar_new)

    # if new calendar is better, update the calendar
    if obj_calendar_new < obj_calendar
        calendar_matrix = calendar_new
        obj_calendar = obj_calendar_new
    end

    return calendar_matrix, obj_calendar, Y_old
end

In [65]:
calendar_matrix = GetCalendarMatrix(calendar, W)

# how many times to repeat this random restart process
repeat = 2 
for count in 1:repeat

    calendar_matrix, obj_calendar, Y_old = random_restart(calendar_matrix, obj_calendar, Y_old)
    
end

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 600
1234567899.51011121314### size: (20, 20, 3, 7)### len: 8
### len: 21
67

LoadError: MethodError: no method matching ComputeObjectiveValueMatrix(::Matrix{Float64})
[0mClosest candidates are:
[0m  ComputeObjectiveValueMatrix(::Any, [91m::Any[39m) at In[64]:2